In [3]:
import numpy as np
import matplotlib.pyplot as plt
from helper_methods import get_train_val, create_dataset, show_images, load_and_transform_image

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
from torchvision.models import ResNet18_Weights
from torch.utils.data import TensorDataset, DataLoader
from transformers import AutoImageProcessor, ViTForImageClassification
from transformers import ViTImageProcessor

In [2]:
# Step 1: Uninstall existing libraries
#pip uninstall transformers accelerate -y

# Step 2: Install required libraries
#pip install transformers[torch] accelerate

# Restart the runtime manually via the "Runtime" menu or by running:
import os
os.kill(os.getpid(), 9)

: 

In [4]:
df_train, df_val = get_train_val(filepath='C:\\Users\\alexa\\Documents\\Deep Learning\\Project\\datasets\\annotations\\annotations\\trainval.txt', val_size=0.2)
# C:\\Users\\alexa\\Documents\\Deep Learning\\Project\\datasets\\images\\images\\

X_train, Y_train = create_dataset(df_train, base_path='C:\\Users\\alexa\\Documents\\Deep Learning\\Project\\datasets\\images\\images\\')
X_val, Y_val = create_dataset(df_val, base_path='C:\\Users\\alexa\\Documents\\Deep Learning\\Project\\datasets\\images\\images\\')

Y_train = Y_train.long()
Y_val = Y_val.long()

In [5]:
labels = torch.unique(Y_train)
print(labels)
print(labels.tolist())

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
        36])
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36]


In [6]:
pip install dataset

   ---------------------------------------- 0.0/233.4 kB ? eta -:--:--
   - -------------------------------------- 10.2/233.4 kB ? eta -:--:--
   ----- ---------------------------------- 30.7/233.4 kB 1.4 MB/s eta 0:00:01
   ---------- ---------------------------- 61.4/233.4 kB 550.5 kB/s eta 0:00:01
   ------------- ------------------------- 81.9/233.4 kB 657.6 kB/s eta 0:00:01
   ------------------ ------------------- 112.6/233.4 kB 656.4 kB/s eta 0:00:01
   -------------------- ----------------- 122.9/233.4 kB 554.9 kB/s eta 0:00:01
   -------------------------- ----------- 163.8/233.4 kB 614.4 kB/s eta 0:00:01
   --------------------------------- ---- 204.8/233.4 kB 623.6 kB/s eta 0:00:01
   ------------------------------------ - 225.3/233.4 kB 626.9 kB/s eta 0:00:01
   -------------------------------------- 233.4/233.4 kB 595.2 kB/s eta 0:00:00
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
from datasets import Dataset

print(type(df_train))
print(type(df_val))
train_dataset = Dataset.from_pandas(df_train)
val_dataset = Dataset.from_pandas(df_val)
print(type(train_dataset))
print(type(val_dataset))
print(train_dataset)


<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'datasets.arrow_dataset.Dataset'>
<class 'datasets.arrow_dataset.Dataset'>
Dataset({
    features: ['image_id', 'class_id', 'species', 'breed', '__index_level_0__'],
    num_rows: 2944
})


In [7]:
model_name_or_path = 'google/vit-base-patch16-224-in21k'
from transformers import ViTForImageClassification
labels = torch.unique(Y_train)
labels = labels.tolist()
model = ViTForImageClassification.from_pretrained(
    model_name_or_path,
    num_labels=torch.max(Y_train) + 1,
    id2label={str(i): c for i, c in enumerate(labels)},
    label2id={c: str(i) for i, c in enumerate(labels)}
)


c:\Users\alexa\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


ViTForImageClassification(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_features=7

In [8]:
from transformers import ViTImageProcessor
processor = ViTImageProcessor.from_pretrained(model_name_or_path)

def transform(example_batch):
    #print(example_batch) # Normally we want a tensor of 'pixel_values': tensor and 'labels': AS INPUTS
    # Take a list of PIL images and turn them to pixel values
    #inputs = processor([x for x in example_batch['image']], return_tensors='pt')
    images_tensors = []
    #Y = []
    base_path = "C:\\Users\\alexa\\Documents\\Deep Learning\\Project\\datasets\\images\\images\\"
    print("Example batch: ", example_batch)
    for im_id in example_batch['image_id']:
        #print("APOEL: ", im_id)
        #image_id, species_id = row[0], row[2]
        image_path = f"{base_path}{im_id}.jpg"  # Adjust format as needed
        image_tensor = load_and_transform_image(image_path)
        images_tensors.append(image_tensor)
    #    Y.append(class_id)
    
    # Stack all tensors to create a single tensor
    inputs = {}
    inputs['pixel_values'] = torch.stack(images_tensors)
    # Don't forget to include the labels!
    #print(example_batch)
    inputs['labels'] = example_batch['class_id']
    #print(inputs)
    return inputs
prepared_df_train = train_dataset.with_transform(transform)
prepared_df_val = val_dataset.with_transform(transform)


In [9]:
#collate_fn,compute_metrics,prepared_ds["train"],prepared_ds["validation"],processor
#from transformers import ViTImageProcessor
#model_name_or_path = 'google/vit-base-patch16-224-in21k'
#processor = ViTImageProcessor.from_pretrained(model_name_or_path)
#def transform(example_batch):
#    # Take a list of PIL images and turn them to pixel values
#    inputs = processor([x for x in example_batch['image']], return_tensors='pt')
#    # Don't forget to include the labels!
#    inputs['labels'] = example_batch['labels']
#    return inputs
#prepared_ds = ds.with_transform(transform)

def collate_fn(batch):
    return {
        'pixel_values': torch.stack([x['pixel_values'] for x in batch]),
        'labels': torch.tensor([x['labels'] for x in batch])
    }
from datasets import load_metric
metric = load_metric("accuracy")
def compute_metrics(p):
    return metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids)


C:\Users\alexa\AppData\Local\Temp\ipykernel_2944\2602130997.py:19: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")
c:\Users\alexa\AppData\Local\Programs\Python\Python310\lib\site-packages\datasets\load.py:759: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [10]:
import transformers
import accelerate

print(f"Transformers version: {transformers.__version__}")
print(f"Accelerate version: {accelerate.__version__}")

Transformers version: 4.40.2
Accelerate version: 0.30.1


In [11]:

from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="C:\\Users\\alexa\\Documents\\Deep Learning\\Project\\outputs",
  per_device_train_batch_size=2, # 16
  evaluation_strategy="steps",
  num_train_epochs=4,
  fp16=True, # True
  #fp16=torch.cuda.is_available(),


  save_steps=100,
  eval_steps=100,
  logging_steps=10,
  learning_rate=2e-4,
  save_total_limit=2,
  remove_unused_columns=False,
  push_to_hub=False,
  report_to='tensorboard',
  load_best_model_at_end=True,

  

)
  

In [12]:
#CUDA_LAUNCH_BLOCKING=1

from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    # <class 'datasets.arrow_dataset.Dataset'> should be given while we give <class 'pandas.core.frame.DataFrame'>
    #train_dataset=prepared_ds["train"], 
    train_dataset=prepared_df_train, #df_train, train_dataset

    #eval_dataset=prepared_ds["validation"],
    eval_dataset=prepared_df_val, #df_val, val_dataset
    tokenizer=processor,
)


In [ ]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"

torch.cuda.empty_cache()
train_results = trainer.train()
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()
#metrics = trainer.evaluate(prepared_ds['validation'])
metrics = trainer.evaluate(prepared_df_val) #df_val

trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)